In [90]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib    
import pandas as pd
import scipy as sc
import csv
import math
import random
from sklearn.model_selection import train_test_split
from scipy.sparse import isspmatrix, csc_matrix, csr_matrix
from scipy.sparse.linalg import eigsh, svds
from numpy.linalg import svd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import io

In [2]:
from skopt import gp_minimize
from skopt.space import Real, Integer

In [3]:
eps                  = np.finfo(float).eps

In [4]:
def createList(r1,r2):
    return [item for item in range(r1,r2 + 1)]

In [5]:
def userSim(Y): 
    print("In userSim")
    N,M              = Y.shape  
    
    Y01              = Y.copy()
    Y01[Y01>0]       = 1
    
    tmp              = (Y**2)@(Y01.T)
    tmp              = tmp + tmp.T
    tmp              = tmp - 2*(Y@(Y.T))
    
    simScore         = 1.0/(1 + np.sqrt(tmp))
    
    nominator        = Y01@Y01.T
    
    denominator      = np.sum(Y01,axis=1).reshape(-1,1)
    denominator      = np.matlib.repmat((denominator),1,N)
    denominator      = denominator + denominator.T
    denominator      = denominator - nominator
    
    B1               = nominator/denominator
    B1[np.isnan(B1)] = 0
    
    simScore         = simScore*B1
    return simScore

In [91]:
def evecs(A,nEvecs):
    npix             = A.shape[0]
    useSparse        = isspmatrix(A)
    
    dd               = 1/((np.sum(A,axis=0))+eps)
    dd               = np.sqrt(dd)

    if(useSparse):
        DD           = sc.sparse.diags(dd)
    else:
        DD           = np.diag(dd)
    
    L                = DD@A@DD
    
    if(useSparse==1):
        ss,v         = eigsh(L,nEvecs,sigma=1)
        ss           = np.flip(ss)
        V            = np.flip(v,axis=1)
    else:
        u,ss,v       = svd(L)
        V            = u[:, :nEvecs]
        
    ss               = ss.reshape(-1,1)
    ss               = ss[:nEvecs]
    
    return V,ss

In [92]:
def spectralClu(A=None,K=None): 
    # A: Affinity Matrix, Higher value -> more similar
    # K: Number of CLuster
    n                = A.shape[0]
    
    D                = np.sum(A,axis=1).reshape(-1,1)
    D[D==0]          = eps
    D                = sc.sparse.spdiags((1.0/np.sqrt(D)).T,0,n,n)
    
    L                = D@A@D
    V,evals          = evecs(L,K)
    
    Vnorm            = (np.linalg.norm(V,axis=1) + eps).reshape(-1,1)
    
    V                = V/Vnorm

    kmeans           = KMeans(n_clusters=K,max_iter=maxiter,random_state=0).fit(V)
    idx              = kmeans.predict(V)

    return idx

In [8]:
def aggregratePrediction(X,UGidx):
    n,m                  = X.shape
    K1                   = len(np.unique(UGidx))
    ctmp_d               = np.ones(n)
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))

    GFreq                = np.bincount(UGidx.reshape(-1,)).reshape(-1,1)
    XPerA                = (ctmp@X)/np.matlib.repmat((GFreq),1,m)
    
    
    return XPerA

In [9]:
def MAE(a,b):
    return np.sum(abs(a*(b!=0) - b))/np.sum(b!=0)

In [10]:
def RMSE(X,Y):
    tmp                        = ((Y - X)*(Y!=0))**2
    
    return math.sqrt(np.sum(tmp)/np.sum(Y!=0))

In [11]:
def AUC(Ypre,Yt,cutoff):
    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    auc                        = 0
    uWithPair                  = 0
    for user in range(N):
        userRating             = Yt[user,:]
        userPrediction         = Ypre[user, :]
        ratedIdx               = userRating>0
        binUserRating          = 2*(userRating[ratedIdx]>=cutoff)-1
        predForKnown           = userPrediction[ratedIdx]
        
        posIdx                 = np.where(binUserRating==1)
        posIdx                 = posIdx[0].T

        negIdx                 = np.where(binUserRating==-1)
        negIdx                 = negIdx[0]
        
        if(np.size(posIdx)!=0 and np.size(negIdx)!=0):
            negIdxGrid         = np.matlib.repmat(negIdx,len(posIdx),1)
            ttlNeg             = np.size(negIdxGrid)
            negIdxGrid         = negIdxGrid.reshape(ttlNeg,1,order='F')
            posIdxGrid         = np.matlib.repmat(posIdx.reshape(-1,1),len(negIdx),1)
            pairs              = np.concatenate((negIdxGrid,posIdxGrid),axis=1)
            pairsPred          = np.concatenate((pairs,predForKnown[negIdxGrid],predForKnown[posIdxGrid]),axis = 1)
            pairsPredWithScore = np.concatenate([pairsPred,(0.5*(pairsPred[:,2]==pairsPred[:,3]).reshape(-1,1)),(1*(pairsPred[:,2]<pairsPred[:,3]).reshape(-1,1))],axis=1)
            auc                = auc + (np.sum(pairsPredWithScore[:,4:6]))/pairsPredWithScore.shape[0]
            uWithPair          = uWithPair + 1    
    auc                        = auc/uWithPair
    
    return auc

In [12]:
def precAtK(Ypre,Yt,k,cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    prec                       = np.zeros((1,k))

    for user in range(N):
        userRating             = Yt[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        userPrediction         = Ypre[user, :]
        prediction             = userPrediction[ratedIdx]
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(min(kNo+1,nz),eps)
            prec[:,kNo]        = prec[:,kNo] + len(intrsct)/mx
            
    prec                       = prec/N
    
    return prec

In [13]:
def recallAtK(Ypre,Yt, k, cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: recall@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    recall                     = np.zeros((1,k))
    
    for user in range(N):
        userRating             = Yt[user, :]
        userPrediction         = Ypre[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        prediction             = userPrediction[ratedIdx]
        
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(len(ratedRelevantIdx),eps)
            recall[:,kNo]      = recall[:,kNo] + len(intrsct)/mx
            
    recall                     = recall/N
    
    return recall 

In [14]:
def ndcgAtk(Ypre,Yt,k):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    res                        = np.zeros((1,k))
    cnt                        = 0
    N,M                        = Yt.shape

    for user in range(N):
        ratedIdx               = Yt[user,:]!=0

        userRating             = Yt[user,ratedIdx]
        userPrediction         = Ypre[user,ratedIdx]
        nz                     = len(userRating)

        ranks                  = np.zeros((1,nz))
        ideal_ranks            = np.zeros((1,nz))
        
        I                      = np.argsort(-userPrediction,axis=0)
        ideal_I                = np.argsort(-userRating,axis=0)

        ranks                  = I
        ideal_ranks            = ideal_I

        oriOrder               = np.argsort(ideal_ranks,axis=0)

        nominator              = userRating/(np.log(ranks+2))
        denominator            = userRating/(np.log(ideal_ranks + 2))
    
        nominator              = nominator[oriOrder]
        denominator            = denominator[oriOrder]
        
        if k > nz:
            nominator          = np.concatenate((nominator, np.zeros((k - nz))))
            denominator        = np.concatenate((denominator, np.zeros((k - nz))))
        elif k < nz:
            nominator          = nominator[0:k]
            denominator        = denominator[0:k]          

        if np.array(np.where(np.cumsum(denominator)== 0)).shape[1] != 0:
            tmp                = np.zeros((1,k))
        else:
            tmp                = np.cumsum(nominator) / np.cumsum(denominator)
            cnt                = cnt + 1
        
        res                    = res + tmp
        
    res                        = res/cnt
    
    return res

In [15]:
def EvaluationAllUpdated(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)
    auc                        = AUC(Yprd, Y1, cutoff)
    precision                  = precAtK(Yprd,Y1, k, cutoff)
    recall                     = recallAtK(Yprd,Y1, k, cutoff)
    f1                         = (2*precision*recall)/(precision + recall + eps)
    ndcg                       = ndcgAtk(Yprd,Y1,k)

    return mae,rmse,auc,precision,recall,f1,ndcg

In [16]:
def EvaluationAllUpdated_N(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)

    return mae,rmse

In [17]:
cols                     = ["Model","MAE","RMSE","AUC","Precision@1","Precision@5","Precision@10","Precision@20",
                            "Precision@30","Precision@40","Recall@1","Recall@5","Recall@10","Recall@20","Recall@30",
                            "Recall@40","F1@1","F1@5","F1@10","F1@20","F1@30","F1@40","NDCG@1","NDCG@5","NDCG@10",
                            "NDCG@20","NDCG@30","NDCG@40"]
Result_Trn               = pd.DataFrame(columns=cols)

In [18]:
cols                     = ["Model","MAE","RMSE"]
Result_Trn_CDR               = pd.DataFrame(columns=cols)
Result_Tst_CDR = pd.DataFrame(columns=cols)

In [19]:
def gradUnifiedLS(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx): 
    
    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    UG                   = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
    VG                   = v[n*d+m*d+K1*d:].reshape(K2,d,order='F')

    UGmatMU              = (UG[UGidx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGidx.reshape(-1,),:]) - V
    
    X                    = U@(V.T)
    Ygt0                 = Y>0
    YMXgt0               = (Y - X)*Ygt0

    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))

    lossobj              = 0
    lossobj              = lossobj + (0.5*(np.sum(YMXgt0**2)))
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2))
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    
    dU                   = dU - YMXgt0@V
    dV                   = dV - (YMXgt0.T)@U

    dU                   = dU - (lambda1*UGmatMU)
    dV                   = dV - (lambda1*VGmatMV)

    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = lambda1*((ctmp)@UGmatMU)
    
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGidx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = lambda1*((ctmp)@VGmatMV)
    
    obj                  = regobj + lossobj
   
    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dUG.flatten('F'),dVG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [20]:
def gradUnifiedMMMF(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGIdx,VGIdx):

    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    theta                = v[n*d+m*d:n*d+m*d+n*(l-1)].reshape(n,l-1,order='F')
    UG                   = v[n*d+m*d+n*(l-1):n*d+m*d+n*(l-1)+K1*d].reshape((K1,d), order='F')
    VG                   = v[n*d+m*d+n*(l-1)+K1*d:n*d+m*d+n*(l-1)+K1*d+K2*d].reshape((K2,d), order='F')
    thetaG               = v[n*d+m*d+n*(l-1)+K1*d+K2*d:].reshape((K1,l-1), order='F')
        
    UGmatMU              = (UG[UGIdx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGIdx.reshape(-1,),:]) - V
    thetaGMtheta         = (thetaG[UGIdx.reshape(-1,),:]) - theta
    
    X                    = U@V.T
    Ygt0                 = Y>0
    BX                   = X*Ygt0
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    dtheta               = np.zeros((n, l-1))
    
    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))
    
    lossobj              = 0
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2) + np.sum(thetaGMtheta**2))
    
    for k in range(l-1):
        S                = Ygt0 - 2*(Y>k+1)
        BZ               = (theta[:,k].reshape(-1,1)@(np.ones([1,m])))*S - BX*S
        lossobj          = lossobj + sum(sum(h(BZ)))
        tmp              = hprime(BZ)*S
        dU               = dU - tmp@V
        dV               = dV - tmp.T@U
        dtheta[:,k]      = tmp@np.ones((m,))
        
    dU                   = dU - lambda1*UGmatMU
    dV                   = dV - lambda1*VGmatMV
    dtheta               = dtheta - lambda1*thetaGMtheta
    
    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGIdx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = np.multiply(lambda1,(ctmp@UGmatMU))
    dthetaG              = np.multiply(lambda1,(ctmp@thetaGMtheta))
 
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGIdx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = np.multiply(lambda1,(ctmp@VGmatMV))
    
    obj                  = regobj + lossobj;            # obj is the objective function that we need to minimize

    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dtheta.flatten('F'),dUG.flatten('F'),dVG.flatten('F'),dthetaG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [21]:
def conjgrad(x0,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx):
    J                     = []
    temp                  = 0
    nu                    = 0.1
    abstol                = 0                                 # stop if gradient magnitude goes below this
    allowNonDecrease      = 0                                 # don't stop if line search fails to find decrease
    digits                = 12                                # digits of precision to use for objective comparisons
    ogfun                 = objGrad
    ogcalls               = 0
    x                     = x0
    numiter               = 0
    j                     = 0
    alpha                 = 10**(-10)
    ogcalls               = ogcalls + 1
    obj,dx,lossobj,regobj = ogfun(x,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    r                     = -dx
    s                     = r
    dirn                  = s
    deltanew              = (r.T)@dirn
    deltazero             = deltanew
    
    while ((numiter<maxiter) and (abs(deltanew)>tol*tol*abs(deltazero)) and (abs(deltanew)>abstol)):
        numiter           = numiter + 1
        j                 = j + 1
        print('\n %.4f' %obj)
        J.append(obj)
        prevobj           = obj

        if (alpha < 10**(-10)):
            alpha         = 10**(-10)

        alpha,obj,dx,ogc  = cgLineSearch(x,obj,dx,dirn,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lossobj,regobj)
        ogcalls           = ogcalls + ogc
        
        temp              = temp + alpha
        x                 = x + alpha*dirn
        r                 = -dx
        deltaold          = deltanew
        deltamid          = r.T@s
        deltanew          = r.T@r
        beta              = (deltanew - deltamid) / deltaold
        dirn              = r + max(0, beta)*dirn
        if ((deltamid/deltanew >= nu) or (dirn.T@dx >= 0)):
            dirn          = r
            j             = 0
        s                 = r
        print(numiter)
        
    return x,numiter,ogcalls,J

In [22]:
def cgLineSearch(x0,obj0,dx0,direction,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lobj,robj):
    seciter                   = 5                               # maximum number of quadratic interpolation iterations
    alpha0                    = alpha
    c1                        = 10**(-4)                        # required decrease in objective (relative to gradient)
    digits                    = 12                              # digits of precision to use for objective comparisons
    gamma                     = 10
    ogfun                     = objGrad
    
    if (alpha0 <= 0):                                           # check initial values
        print('alpha0 must be greater than zero')
    
    obj                       = obj0                            # begin line search
    dx                        = dx0
    etazero                   = (dx.T)@direction
    etaprev                   = etazero
    alpha                     = alpha0
    ogcalls                   = 0
    lossobj                   = lobj
    regobj                    = robj
    
    alpha,obj,dx,lossobj,regobj,ogcalls = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    
    obj,dx,lossobj,regobj     = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    ogcalls                   = ogcalls + 1
    
    oldalpha,oldobj,olddx                           = saveAlpha(alpha,obj,dx)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)

    beta                      = alpha
    eta                       = dx.T@direction
    i                         = 0
    
    while((abs(eta)>c2*abs(etazero)) and (i<seciter) and (pround(obj,digits) <= pround(obj0,digits)) and (etaprev!=eta) and (np.sum((x0 + alpha*direction) != x0)>0)):
        beta                  = eta*beta / (etaprev - eta)
        oldalpha,oldobj,olddx = saveAlpha(alpha,obj,dx)
        alpha                 = alpha + beta
        if(alpha<=0):
            alpha             = 1
        etaprev               = eta
        i                     = i + 1
        obj,dx,lossobj,regobj = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        eta                   = dx.T@direction

    alpha,obj,dx,lossobj,regobj,ogcalls             = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)
    checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction)
    
    return alpha,obj,dx,ogcalls 

In [23]:
def findNonZeroAlpha(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj):
    if(np.array_equal((x0+alpha*direction),x0)):                # Make sure alpha isn't smaller than level of precision
        preAlpha              = alpha
        
        while(np.array_equal((x0+alpha*direction),x0)):
            alpha             = alpha*gamma
        
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        
    return alpha,obj,dx,lossobj,regobj,ogcalls

In [24]:
def backtrack(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx):
    #ensures either (1) Armijo lowex0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcallsr objective, or (2) infinitesimal alpha
    doBacktrack               = 0
    preAlpha                  = alpha
    while((pround(obj,digits)>pround(obj0 + c1*alpha*etazero, digits)) and (np.sum((x0+alpha*direction)!=x0)>0)):
        if(ogcalls > 1):
            if((oldalpha > alpha/gamma) and (oldobj < (obj0+c1*oldalpha*etazero))):
                alpha,obj,dx  = restoreAlpha(oldalpha,oldobj,olddx)
                doBacktrack   = 1
                break
        alpha                 = alpha/gamma
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        doBacktrack           = 1
    return alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack

In [25]:
def saveAlpha(alpha,obj,dx):
    #make sure current step yields decrease in objective
    oldalpha   = alpha
    oldobj     = obj
    olddx      = dx
    return oldalpha,oldobj,olddx

In [26]:
def restoreAlpha(oldalpha,oldobj,olddx):
    alpha      = oldalpha
    obj        = oldobj
    dx         = olddx
    return alpha,obj,dx

In [27]:
def pround(x,d):
    d          = round(d)
    if (d<1):
        print('Number of digits must be integer d = %.4e \n'%d)
    if(x==0 or math.isnan(x) or math.isinf(x)):
        y      = x   
    else:
        p      = math.floor(math.log10(abs(x)))+1
        factor = 10**(d-p)
        y      = np.round(x*factor)/factor
    return y

In [28]:
def checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction):
    if round(obj, digits) >= round(obj0, digits):
        print('Warning: Finished line search without decreasing objective.\nMay have reached limit of precision, or obj/grad code may be broken.\n')
    if etazero == eta:
        print('Warning: Line search yielded no change in directional derivative.\nMay have reached limit of precision.\n')
    if sum(x0 + alpha * direction != x0) == 0:
        print('Warning: Line search yielded no change in position.\nMay have reached limit of precision.\n')

In [29]:
def h(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01/2 - zin01*z + zin01*(z**2)/2 + zle0/2 - zle0*z
    return ret  

In [30]:
def hprime(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01*z - zin01 - zle0
    return ret

In [31]:
def m3fSoftmax(xy,theta):
    n,m        = xy.shape
    n1,l1      = theta.shape
    if(n!=n1):
        print('sizes of xy and theta don''t match');
    y          = np.ones((n,m))
    for i in range(l1):
        tmp    = ((theta[:,i]).reshape(-1,1))@(np.ones((1,m)))
        tmp    = xy>=tmp
        y      = y + tmp
    return y

## Preprocessing Video Games domain

In [32]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [33]:
n_users_video = df.user_id.nunique(dropna = True)
m_items_video = df.game_id.nunique(dropna = True)
print(str(n_users_video) + " " + str(m_items_video))

826767 50210


In [34]:
user_ids_video = df.user_id.unique()
item_ids_video = df.game_id.unique()
print(user_ids_video.shape)
print(item_ids_video.shape)

(826767,)
(50210,)


## Preprocessing Movies and TV domain

In [35]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [36]:
n_users_mt = df.user_id.nunique(dropna = True)
m_items_mt = df.movie_id.nunique(dropna = True)
print(str(n_users_mt) + " " + str(m_items_mt)) 

2088620 200941


In [37]:
user_ids_mt = df.user_id.unique()
item_ids_mt = df.movie_id.unique()
print(user_ids_mt.shape)
print(item_ids_mt.shape)

(2088620,)
(200941,)


## Dataset trimming

In [241]:
overlap = 3000

In [242]:
print(user_ids_video.shape)
print(user_ids_mt.shape)

(826767,)
(2088620,)


In [243]:
overlapping_user_ids = np.intersect1d(user_ids_video, user_ids_mt)
print(overlapping_user_ids)
print(overlapping_user_ids.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 ... 'AZZW55XIQD8QC' 'AZZY72H9Y2F6O' 'AZZZ9DDUPKNKC']
(155797,)


In [244]:
non_overlapping_user_ids_video = np.array(list(set(user_ids_video.tolist()).difference(set(overlapping_user_ids.tolist()))))
non_overlapping_user_ids_mt = np.array(list(set(user_ids_mt.tolist()).difference(set(overlapping_user_ids.tolist()))))
print(non_overlapping_user_ids_video.shape)
print(non_overlapping_user_ids_mt.shape)

(670970,)
(1932823,)


In [245]:
req_overlap_user_ids = np.random.choice(overlapping_user_ids, size=overlap, replace=False)
print(req_overlap_user_ids.shape)
# test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

(3000,)


In [246]:
req_non_overlap_user_ids_video = np.random.choice(non_overlapping_user_ids_video, size= 9000 - overlap, replace=False)
req_non_overlap_user_ids_mt = np.random.choice(non_overlapping_user_ids_mt, size= 7000 - overlap, replace=False)
print(req_non_overlap_user_ids_video.shape)
print(req_non_overlap_user_ids_mt.shape)

(6000,)
(4000,)


In [247]:
new_user_ids_video = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_video), axis=None)
new_user_ids_mt = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_mt), axis=None)
np.random.shuffle(new_user_ids_video)
np.random.shuffle(new_user_ids_mt)
print(new_user_ids_video.shape)
print(new_user_ids_mt.shape)

(9000,)
(7000,)


In [248]:
print(new_user_ids_video)
print(new_user_ids_mt)

['A2YTOQSJKJFWCM' 'A231DX7CGG47CW' 'A2TIORV4AXKCJ1' ... 'A2UCOWQDX7NJGV'
 'A2SYNL4YX73KNY' 'A2FP8WCVI11J6P']
['A52EFRFBDWU1F' 'A218LJH6RPETU' 'AAQ75C4BJUQLO' ... 'A1D6AMH6WH4DBW'
 'A18WZHOI791MAP' 'A2GDIPZYDPTXL0']


In [249]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video, new_user_ids_mt)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A04665232QBWBWC3DQOXI' 'A05299163KOLQAXMGQF3W' 'A102QYFYRH664Z' ...
 'AZXHFBHEVY5PT' 'AZXP46IB63PU8' 'AZY5NA6OWM559']
(3000,)


## Creating rating matrix for Video Games domain

In [250]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [251]:
df = df.loc[df['user_id'].isin(new_user_ids_video.tolist())] 

In [252]:
df.shape

(15293, 4)

In [253]:
present_rating = df.rating.unique()
print(present_rating)

[4. 5. 3. 1. 2.]


In [254]:
new_item_ids_video = df.game_id.unique()
print(new_item_ids_video.shape)
print(new_item_ids_video)

(7779,)
['0439573947' '0545076412' '0700026657' ... 'B00KFQ586U' 'B00KGGJPX6'
 'B00KKAQYXM']


In [255]:
print(item_ids_video.shape)

(50210,)


In [256]:
item_ids_video_rated_unrated = new_item_ids_video
np.random.shuffle(item_ids_video_rated_unrated)
print(item_ids_video_rated_unrated.shape)

(7779,)


In [257]:
rating_matrix_video = np.zeros([9000, item_ids_video_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_video.shape)

(9000, 7779)


In [258]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_video:
        rating_matrix_video[np.where(new_user_ids_video == df['user_id'][ind])[0][0]][np.where(item_ids_video_rated_unrated == df['game_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['game_id'][ind]) + " " + str(df['rating'][ind]))

A366EUKI8WMGYB 0439573947 4.0
A1VJVU8YSBPAY7 0545076412 4.0
A2VLH0R0UP9ULJ 0700026657 5.0
AV969NA4CBP10 0700026657 5.0
AD4IGRSR4TGF0 0700099867 4.0
A1QQELOD9SMEFT 9625990674 4.0
A3JD0Y1BKA2EUV 9625991921 4.0
A3AL3WLSX0RSTC 9625992421 3.0
A3GH9EPA8423L1 9861019731 5.0
A2VIQLU57RMFGG 9862561165 5.0
AIJ2BICYGA6Y7 9882106463 5.0
A2KRP1OQ3SKGT2 9882106463 1.0
AZHHJAQAD8BCJ 9882155456 5.0
AY0KVFDA8U2S3 9882155456 5.0
A1FYY9K6LNPRH7 988800171X 5.0
A21W3IFHWFAHGD B000006OVE 5.0
A2K7B6UNB70V0Q B000006OVE 5.0
A3EU0FX5OZZ32P B000006OVE 3.0
A26A9FFNCGQVDF B000006OVI 4.0
A22R21KQ28HAB4 B000006OVJ 4.0
A18EZ7C2K9BN1P B000006OWT 1.0
A3HSIXJL85N8UK B000009QCW 5.0
AM8K619L97JWR B000009QCW 5.0
AAQ75C4BJUQLO B000009QCX 4.0
A1LMKIEKF566AU B000009QD1 5.0
A1RDAKYE0Y8IRJ B00000DMA8 5.0
A3K64PCEG21Q6J B00000DMAD 5.0
A1878O39ZKG24F B00000DMAG 5.0
ANHTCYJSBE0QC B00000DMAH 5.0
A37HIFUT0LEF2C B00000DMAN 5.0
A3VBAQSNYX343S B00000DMAP 4.0
A35BLWDHM3M5R6 B00000DMAQ 5.0
AMAZ9S2WM69FK B00000DMAQ 1.0
A2N1C9JKI2C5XD B000

A37T9V996O3RAS B00009WAVI 5.0
A3NN4RTUN0LHBN B00009WAVL 4.0
A2WQJDLGWLQN7X B00009WAVL 5.0
A3A8ZKXM2UL4E9 B00009WAW5 5.0
A3MC73CY4KUNA0 B00009WAW7 4.0
A2WJUMA1LYP586 B00009WNZA 5.0
A15PXE4TK0KN7K B00009WNZA 5.0
A42FZU5QJB9LK B00009WNZA 5.0
A1R915SCB7UQLI B00009WNZA 5.0
A1JOCFRO3UC0QE B00009WNZA 5.0
A13LSLOS3O7AV8 B00009WNZA 5.0
ALGX646JKT4KG B00009WNZA 5.0
A1I75CZO2D324A B00009X3UY 4.0
A2Q365MMU5N3JC B00009X3V2 1.0
API7UGE1NBDZF B00009X3VB 5.0
A3CTEM06O5EK5U B00009X3VB 2.0
A14F106N1M9AAQ B00009X3VD 3.0
A3PM4B0GKF6ULY B00009X3VF 4.0
A3JQFLLGHUCSC B00009X7DN 1.0
AK02BT6CWTQFM B00009XS6D 1.0
AAQ75C4BJUQLO B00009XS6G 2.0
A3NN4RTUN0LHBN B00009YEJY 4.0
ABVHVW0G2AKE1 B00009YEJY 5.0
A1IEIWUCTWZ59M B00009YEK0 5.0
A11RJH8JB8HYJH B00009YEK0 5.0
A2QKYUBBCRFWJB B00009YEK0 1.0
A3NN4RTUN0LHBN B00009YEK1 2.0
A2HVLXUPN3B3OI B00009YEK1 5.0
A2TKEQG9SU8BYA B00009YEK5 3.0
A1VFJ2AOGZUTSW B00009YEK6 4.0
A3NN4RTUN0LHBN B00009YEK6 1.0
A12CC4FTM92QQY B00009YFU5 5.0
A2ECVA2TTVTOTC B00009YXIE 4.0
A1Y09QR1KE0JXB B0

In [259]:
rating_matrix_video

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [260]:
unique, frequency = np.unique(rating_matrix_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [69995707     1715      892     1449     3041     8196]


## Creating rating matrix for movies and TV domain

In [261]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [262]:
df = df.loc[df['user_id'].isin(new_user_ids_mt.tolist())] 
df.shape

(20013, 4)

In [263]:
new_item_ids_mt = df.movie_id.unique()
print(new_item_ids_mt.shape)
print(new_item_ids_mt)

(11687,)
['0005119367' '0307142485' '0307142493' ... 'B00KPYT3LS' 'B00L4IDS4W'
 'B00LCAD24S']


In [264]:
print(item_ids_mt.shape)

(200941,)


In [265]:
item_ids_mt_rated_unrated = new_item_ids_mt
np.random.shuffle(item_ids_mt_rated_unrated)
print(item_ids_mt_rated_unrated.shape)

(11687,)


In [266]:
rating_matrix_mt = np.zeros([7000, item_ids_mt_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_mt.shape)

(7000, 11687)


In [267]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_mt:
        rating_matrix_mt[np.where(new_user_ids_mt == df['user_id'][ind])[0][0]][np.where(item_ids_mt_rated_unrated == df['movie_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['movie_id'][ind]) + " " + str(df['rating'][ind]))

A1O7D6U9SO44XN 0005119367 5.0
A8SO8IEX47DB9 0307142485 5.0
A3FBFQ2P1D5YVU 0307142493 5.0
A1U7TAQ49IX3LP 0307142493 5.0
A3TBZFU0O75YCL 0307514161 5.0
A2GIUIYGU8RE0M 0310256542 5.0
ANHTCYJSBE0QC 0310263662 5.0
A23Y1GZBYKQFCU 0310263662 5.0
AWXM455BNL2RM 0310263662 3.0
A2F1P10GGZ7K2T 0310263662 5.0
A2Q5A0L5RVUJ17 0310263662 5.0
AOGUS6N3KP2O5 0310263662 5.0
A2DSN56RQA699G 0310263662 2.0
A2R4XG0L7Z01V8 0310263662 4.0
A3RLEKKHVJBB5L 0310263662 3.0
A3A52DFJRSSUZ4 0310263662 5.0
AUDSM2CTLLW1Q 0310263662 5.0
A3BWEZPRELE5FS 0310263662 4.0
A2N1B7LYCAG14W 0310263662 5.0
A7LL11KVEYFYA 0310263662 5.0
A2J9COKR9JAZUQ 0310263662 5.0
A3C86O2NNH36U4 0310269156 5.0
A3C86O2NNH36U4 0310269164 5.0
A1IAGMAAZLXENR 0310271169 5.0
A16LVBJ7IMKEZR 0310274281 5.0
A81UEAAE3CSHZ 0310274281 4.0
A2VBJPBMO0HBTN 0310285569 5.0
A1WEOBZXS7JVA9 0323056946 3.0
AX3NVXGCTQ8AN 0510539610 5.0
AUDSM2CTLLW1Q 0615115187 4.0
A3OQHQ73ZWUBFA 061524226X 5.0
ANWD9ZLK5Z8K7 0718000315 5.0
A1L5EWG2UZCVK3 0736915842 3.0
A3TL0Z2D5VRKHA 07369

A2SWDZLD8D337B 0783241569 5.0
AFVI6OICZFKYF 0783241593 4.0
A1CLHLW9PFKG9Q 0783241607 3.0
AHPCWHAKOVZXS 0783241607 5.0
AFVI6OICZFKYF 0783241917 3.0
A1PGC1GEBY9CZU 0783241917 4.0
A2R4XG0L7Z01V8 0783241917 4.0
A2R4XG0L7Z01V8 0783242263 1.0
A2GANR9I6XHTU9 0783243456 2.0
A2GANR9I6XHTU9 0783243499 5.0
A1IJC479QLA9F3 0783243499 1.0
AFVI6OICZFKYF 0783243499 4.0
A1JKFUUCZRA3Z4 0783243499 2.0
A3D2NYBUA8W3VY 0783243499 5.0
AKV6RO0RWFEIF 0783243499 2.0
AV5G37VFE5NVD 0783243499 5.0
AP54641ISD9HW 0783582706 5.0
A3BQ4AIDQB5NI5 0784001804 5.0
A2R4XG0L7Z01V8 078400255X 2.0
A3F8FYSXRQNNZT 0784010188 5.0
A3P1EB052N9Y90 0784010188 1.0
AQCU0K635E3ER 0784010188 1.0
A24WMBO3LUVUZR 0784010188 5.0
A2R4XG0L7Z01V8 0784010188 4.0
A2RC56G29816HL 0784010188 5.0
A31R7SO1NATXVZ 078401020X 1.0
A2V0KJCZWW6VL7 078401020X 5.0
A2R4XG0L7Z01V8 078401020X 4.0
ALM3XUE5HFLCK 078401020X 4.0
A10JA3E0YEQIJ4 0784010218 5.0
A2LEXNAIKW9P8Z 0784010218 5.0
A309YE3EFAOE2X 0784010218 5.0
A3UD9MCTF54GYP 0784010218 5.0
A2R4XG0L7Z01V8 0784

In [268]:
unique, frequency = np.unique(rating_matrix_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [81788987     1377     1060     2119     3865    11592]


In [32]:
from pandas import read_csv

In [95]:
rating_matrix_video = np.loadtxt("rating_matrix_video_1k.csv", delimiter=",")

In [96]:
temp_d = read_csv('new_user_ids_video_1k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_video= temp_np.reshape(temp_np.shape[0],)

In [97]:
temp_d = read_csv('item_ids_video_1k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_video_rated_unrated = temp_np.reshape(temp_np.shape[0],)

In [98]:
rating_matrix_mt = np.loadtxt("rating_matrix_mt_1k.csv", delimiter=",")

In [99]:
temp_d = read_csv('new_user_ids_mt_1k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_mt = temp_np.reshape(temp_np.shape[0],)

In [100]:
temp_d = read_csv('item_ids_mt_1k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_mt_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Removing overlapping test user_ids from target domain

In [101]:
d                        = 100 

In [102]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video, new_user_ids_mt)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 'A00254261GWP2LJ0A209G' 'A00274963RTZUW5BU5ROI' 'A00472881KT6WR48K907X'
 'A0061198154UGJDK69CYW' 'A00674852ZTZ1WDKI8P68' 'A00794212LQPFSVO1ZAOZ'
 'A00875592Z7LISWIUI55S' 'A00991433TUPZ1NQ6XL27' 'A01007512W8LIXGVKI7HZ'
 'A01078113CVG36M2OQKNM' 'A0108605182MOPK0I9YBV' 'A012468118FTQAINEI0OQ'
 'A0139288TEE6WV1DPN92' 'A016324216JJ0ALNHZ9YX' 'A01632683TJ1GCADQ8B7X'
 'A01731241L8XB77IR452F' 'A017699216H6YAFBGYJOW' 'A01834923U8SVNBY3SKLY'
 'A023023334QO4HEJJIFCP' 'A023090719X7MTBCLM19B' 'A02343532EER409UDAGA8'
 'A0250634ZVA1JL249ZFF' 'A02755422E9NI29TCQ5W3' 'A02857423RPUOIY7KBDA7'
 'A029813613XQMSQO8N3LA' 'A031533437UL5KXSH8FNB' 'A03382832SKK7Q3UMJ6L1'
 'A034294113MZYOJ6UMXUM' 'A03550173UTIP5TNQ9T7P' 'A036041773RWH8ILUKBS'
 'A036147939NFPC389VLK' 'A03635112KI7HHB868OAE' 'A036513549TVB6QSFK04'
 'A036815118EINHQSC2UXR' 'A0369367M6U3LSLAB3C1' 'A03781073B1MY5I79GX2O'
 'A0402564TCEO67AUZFJO' 'A04307562LMGFZCANFC2P' 'A04418443

In [103]:
#Change as per Overlap count
#train = 2500 when overlap = 3000
#train = 800 when overlap = 1000
#train = 5000 when overlap = 6000
train = np.random.choice(overlapping_user_ids_t, size=800, replace=False)
test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

In [104]:
X_train = np.empty((0,100))
Y_train = np.empty((0,100))
X_test = np.empty((0,100))
Y_test = []

In [105]:
print(rating_matrix_mt.shape)

(7000, 10542)


In [106]:
for user_id in test:
    ind1 = np.where(new_user_ids_mt == user_id)
    Y_test.append(rating_matrix_mt[ind1][0])
    new_user_ids_mt =np.delete(new_user_ids_mt, ind1, 0)
    rating_matrix_mt = np.delete(rating_matrix_mt, ind1, 0)

In [107]:
Y_test = np.array(Y_test)
print(Y_test.shape)

(200, 10542)


In [108]:
print(new_user_ids_mt.shape)
print(rating_matrix_mt.shape)

(6800,)
(6800, 10542)


## GRS for Video Games domain

In [109]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                              #Number of Clusters in User Space
K2                       = 22                              #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 1
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [110]:
n, m = rating_matrix_video.shape
print(n, m)

9000 7045


In [111]:
#Spectral Clustering
simU                     = userSim(rating_matrix_video)
simU                     = (simU + simU.T)/2
UGidx_video              = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_video.T)
simI                     = (simI + simI.T)/2
VGidx_video              = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


In [50]:
print(UGidx_video)
print(UGidx_video.shape)

[[8]
 [6]
 [8]
 ...
 [8]
 [8]
 [8]]
(9000, 1)


In [51]:
print(VGidx_video)
print(VGidx_video.shape)

[[17]
 [ 3]
 [10]
 ...
 [20]
 [20]
 [12]]
(7045, 1)


In [52]:
unique, frequency = np.unique(UGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [ 494  583  493  487  476  355 4743  375  530  464]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
Frequency Values: [246 247 235 503 250 279 245 291 551 243 386 352 273 285 302 400 243 468
 278 477 233 258]


### Recommendation

In [ ]:
# np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [ ]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_video,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_video,VGidx_video)
U_video                   = v[0:n*d].reshape(n,d,order='F')
V_video                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_video                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_video@V_video.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_video)
XPerAMat            = XPerA[UGidx_video.reshape(-1,),:]

XPer_Grp                    = UG_video@V_video.T
XPer_GrpMat                 = XPer_Grp[UGidx_video.reshape(-1,),:]
XPer_GrpMat[XPer_GrpMat<=1] = 1
XPer_GrpMat[XPer_GrpMat>=5] = 5

In [ ]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPerAMat, rating_matrix_video, k, cutoff)
ResultTrnULS_Grp_PerA                 = ["Unified_LS_Grp_PerA_Run_2_3k",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Unified_LS_Grp_PerA_Run_2_3k",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [ ]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat, rating_matrix_video, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_Run_2_3k",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Unified_LS_Grp_Run_2_3k",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

## GRS for Movies and TV domain

In [ ]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                               #Number of Clusters in User Space
K2                       = 100                             #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 30
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [ ]:
n, m = rating_matrix_mt.shape
print(n, m)

In [ ]:
#Spectral Clustering
simU                     = userSim(rating_matrix_mt)
simU                     = (simU + simU.T)/2

UGidx_mt                    = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_mt.T)
simI                     = (simI + simI.T)/2

VGidx_mt                    = spectralClu(simI,K2).reshape(-1,1)

In [ ]:
print(UGidx_mt)
print(UGidx_mt.shape)

In [ ]:
print(VGidx_mt)
print(VGidx_mt.shape)

In [ ]:
unique, frequency = np.unique(UGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

### Recommendation

In [ ]:
# np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [ ]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_mt,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_mt,VGidx_mt)
U_mt                  = v[0:n*d].reshape(n,d,order='F')
V_mt                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_mt                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_mt@V_mt.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_mt)
XPerAMat_mt            = XPerA[UGidx_mt.reshape(-1,),:]

XPer_Grp                    = UG_mt@V_mt.T
XPer_GrpMat_mt                 = XPer_Grp[UGidx_mt.reshape(-1,),:]
XPer_GrpMat_mt[XPer_GrpMat_mt<=1] = 1
XPer_GrpMat_mt[XPer_GrpMat_mt>=5] = 5

In [ ]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPerAMat_mt, rating_matrix_mt, k, cutoff)
ResultTrnULS_Grp_PerA                 = ["Unified_LS_Grp_PerA_mt_Run_2_3k",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Unified_LS_Grp_PerA_mt",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [ ]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat_mt, rating_matrix_mt, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_mt_Run_2_3k",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Unified_LS_Grp_mt_Run_2_3k",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [ ]:
Result_Trn

## Creating X, Y matrices for mapping

In [ ]:
#Adding individual latent vectors
for user_id in train:
    source_ind = np.where(new_user_ids_video == user_id)
    target_ind = np.where(new_user_ids_mt == user_id)
    X_train = np.concatenate((X_train, U_video[source_ind]), axis = 0)
    Y_train = np.concatenate((Y_train, U_mt[target_ind]), axis = 0)
    
for user_id in test:
    source_ind = np.where(new_user_ids_video == user_id)
    target_ind = np.where(new_user_ids_mt == user_id)
    X_test = np.concatenate((X_test, U_video[source_ind]), axis = 0)  

    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
groupsToMembers_video = []
groupsToMembers_mt = []
for i in range(K1):
    groupsToMembers_video.append([])
    groupsToMembers_mt.append([])

for i in range(UGidx_video.shape[0]):
    groupsToMembers_video[int(UGidx_video[i][0])].append(new_user_ids_video[i])
    
for i in range(UGidx_mt.shape[0]):
    groupsToMembers_mt[int(UGidx_mt[i][0])].append(new_user_ids_mt[i])
    
print(len(groupsToMembers_video))
print(len(groupsToMembers_mt))

In [ ]:
for i in range(len(groupsToMembers_video)):
    print(len(groupsToMembers_video[i]))

In [ ]:
for i in range(len(groupsToMembers_mt)):
    print(len(groupsToMembers_mt[i]))

In [ ]:
print(groupsToMembers_mt[5])

In [ ]:
counter = 0
for i in range(len(groupsToMembers_video)):
    currGroup_video = groupsToMembers_video[i]
    for j in range(len(groupsToMembers_mt)):
        currGroup_mt = groupsToMembers_mt[j]
        if len(set(currGroup_video).intersection(set(currGroup_mt))) > 0:
            counter = counter + 1
            X_train = np.row_stack((X_train, UG_video[i]))
            Y_train = np.row_stack((Y_train, UG_mt[j]))
            
print(X_train.shape)
print(Y_train.shape)
print(counter)

## MLP Mapping

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
class GetDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(100, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 100)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [ ]:
# Set fixed random number seed
# torch.manual_seed(42)

In [ ]:
dataset = GetDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=X_train.shape[0], shuffle=False, num_workers=0)

In [ ]:
# space = [
#     Real(1e-3, 5e-2, name='lr'),
#     Real(1e-5, 1e-2, name='decay')
# ]

In [ ]:
# def trainMLPAndGetMAE(hyperparameters):
#     # Extract hyperparameters
#     learning_rate = hyperparameters[0]
#     wt_decay = hyperparameters[1]
    
#     mlp = MLP()
  
#     # Define the loss function and optimizer
#     loss_function = nn.MSELoss()
#     optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate, weight_decay=wt_decay)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)
    
#     counter = 0
#     # Run the training loop
#     while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum

#         # Print epoch
#     #     print(f'Starting epoch {epoch+1}')

#         # Set current loss value
#         current_loss = 0.0
#         losses = []
#         # Iterate over the DataLoader for training data
#         for i, data in enumerate(trainloader, 0):

#           # Get and prepare inputs
#     #         if counter > 2:
#     #             break
#     #         print(counter)
#             inputs, targets = data
#             inputs, targets = inputs.float(), targets.float()
#             targets = targets.reshape((targets.shape[0], 100))

#     #         print("Inputs shape: ", inputs.shape)
#     #         print("Target shape: ", targets.shape)
#     #         counter += 1

#           # Zero the gradients
#             optimizer.zero_grad()

#           # Perform forward pass
#             outputs = mlp(inputs)
#     #         print("Output shape: ", outputs.shape)

# #             outputs_np = outputs.detach().numpy()
# #             targets_np = targets.detach().numpy()
#             t_V_mt = torch.from_numpy(V_mt.T).float()

# #             t_vec = targets_np@V_mt.T
# #             o_vec = outputs_np@V_mt.T
#             t_vec = torch.matmul(targets, t_V_mt)
#             o_vec = torch.matmul(outputs, t_V_mt)
#     #         print("t_vec.shape: ", t_vec.shape)
#     #         print("o_vec.shape: ", o_vec.shape)
#             indicator_matrix = (t_vec >= 0.7)
#     #         print("indicator matrix: ", indicator_matrix)
#     #         print(np.all(indicator_matrix == True))
# #             t_vec_interacted = torch.from_numpy(t_vec*indicator_matrix).requires_grad_() 
# #             o_vec_interacted = torch.from_numpy(o_vec*indicator_matrix).requires_grad_() 
#             t_vec_interacted = torch.mul(t_vec, indicator_matrix)
#             o_vec_interacted = torch.mul(o_vec, indicator_matrix)
# #             t = torch.from_numpy(a)

#           # Compute loss
#             loss = loss_function(o_vec_interacted, t_vec_interacted)
#     #         print("Loss: ", loss.item())
#             print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
#             losses.append(loss.item())
#           # Perform backward pass
#             loss.backward()

#           # Perform optimization
#             optimizer.step()


#         mean_loss = sum(losses) / len(losses)
#         scheduler.step(mean_loss)
# #         print(f"Loss at epoch {counter + 1} = {mean_loss}")
#         if optimizer.param_groups[-1]['lr'] <= 1e-4:
#             break;
#         counter += 1

#     # Process is complete.
#     print('Training process has finished.')
    
#     pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
#     print("pred_X_train: ", pred_X_train.shape)
#     par1 = pred_X_train@V_mt.T
#     par2 = Y_train@V_mt.T
#     mae = MAE(par1, par2)
#     print("MAE: ", mae)
#     print()
#     print()
#     return mae

In [ ]:
# def objective(hyperparameters):
#     print(hyperparameters)
#     return trainMLPAndGetMAE(hyperparameters)

In [ ]:
# # Perform Bayesian optimization
# result = gp_minimize(objective, space, n_calls=20)

In [ ]:
# Get the best hyperparameters and performance
# best_hyperparameters = {
#     'K1' = result.x[0]
#     'K2' = result.x[1]
#     'lambda1UnifiedLS' = result.x[2]
#     'lambda3UnifiedLS' = result.x[3]
# }
# best_performance = result.fun
# print("Best MAE: {:.4f}".format(result.fun))
# print("Best Hyperparameters: {}".format(dict(zip(['lr', 'decay'], result.x))))

In [ ]:
# Print the MAE values and hyperparameters for each call
# for i, val in enumerate(result.func_vals):
#     print("Call {}: MAE={:.4f}, Hyperparameters={}".format(i, val, result.x_iters[i]))

In [ ]:
# Print the values of the optimized hyperparameters
# print('Optimized Hyperparameters:')
# for hyperparameter, value in zip(space, result.x):
#     print('{}: {}'.format(hyperparameter.name, value))

## MLP with tuned hyperparameters

In [ ]:
# torch.manual_seed(42)

In [ ]:
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.018113971388685, weight_decay=0.0052151663029590236)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)

In [ ]:
counter = 0
# Run the training loop
while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum
    
    # Print epoch
#     print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    losses = []
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
#         if counter > 2:
#             break
#         print(counter)
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 100))
        
#         print("Inputs shape: ", inputs.shape)
#         print("Target shape: ", targets.shape)
#         counter += 1
      
      # Zero the gradients
        optimizer.zero_grad()
      
      # Perform forward pass
        outputs = mlp(inputs)
#         print("Output shape: ", outputs.shape)
        
#         outputs_np = outputs.detach().numpy()
#         targets_np = targets.detach().numpy()
        t_V_mt = torch.from_numpy(V_mt.T).float()
    
#         t_vec = targets_np@V_mt.T
#         o_vec = outputs_np@V_mt.T
#         print("t_vec.shape: ", t_vec.shape)
#         print("o_vec.shape: ", o_vec.shape)
        t_vec = torch.matmul(targets, t_V_mt)
        o_vec = torch.matmul(outputs, t_V_mt)
        indicator_matrix = (t_vec >= 0.7)
#         print("indicator matrix: ", indicator_matrix)
#         print(np.all(indicator_matrix == True))
#         t_vec_interacted = t_vec*indicator_matrix
#         o_vec_interacted = o_vec*indicator_matrix
        t_vec_interacted = torch.mul(t_vec, indicator_matrix)
        o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

        # Compute loss
        loss = loss_function(o_vec_interacted, t_vec_interacted)
        
#         print("Loss: ", loss.item())
        print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
        losses.append(loss.item())
      # Perform backward pass
        loss.backward()
      
      # Perform optimization
        optimizer.step()
        
        
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    print(f"Loss at epoch {counter + 1} = {mean_loss}")
    if optimizer.param_groups[-1]['lr'] <= 1e-4:
        break;
    counter += 1

# Process is complete.
print('Training process has finished.')

In [ ]:
pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
print(pred_X_train.shape)
cutoff = 3
k = 40
par1 = pred_X_train@V_mt.T
par2 = Y_train@V_mt.T
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Training Metrics")
# print(mae, rmse)

In [ ]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Training_Metrics                = ["CDGRS_Train",mae,rmse]
Result_Trn_CDR.loc[len(Result_Trn_CDR)]       = ["Train-2_3k",mae,rmse]

In [ ]:
pred_X_test = mlp(torch.from_numpy(np.float32(X_test))).detach().numpy()
print(pred_X_test.shape)
cutoff = 3
k = 40
par1 = pred_X_test@V_mt.T
par1[par1 > 5] = 5
par2 = Y_test
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Test Metrics")
# print(mae, rmse)

In [ ]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Testing_Metrics                = ["CDGRS_Test",mae,rmse]
Result_Tst_CDR.loc[len(Result_Tst_CDR)]       = ["Test-2_3k",mae,rmse]

In [ ]:
Result_Trn_CDR

In [ ]:
Result_Tst_CDR